# Setup 

In [2]:
# Setup - Run only once per Kernel App
%conda install openjdk -y

# install PySpark
%pip install pyspark==3.3.0

# restart kernel
from IPython.core.display import HTML
HTML("<script>Jupyter.notebook.kernel.restart()</script>")

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 23.3.1
  latest version: 23.10.0

Please update conda by running

    $ conda update -n base -c defaults conda

Or to minimize the number of packages updated during conda update use

     conda install conda=23.10.0



# All requested packages already installed.


Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [3]:
# Import pyspark and build Spark session
from pyspark.sql import SparkSession

spark = (
    SparkSession.builder.appName("PySparkApp")
    .config("spark.jars.packages", "org.apache.hadoop:hadoop-aws:3.2.2")
    .config(
        "fs.s3a.aws.credentials.provider",
        "com.amazonaws.auth.ContainerCredentialsProvider",
    )
    .getOrCreate()
)

print(spark.version)

:: loading settings :: url = jar:file:/opt/conda/lib/python3.10/site-packages/pyspark/jars/ivy-2.5.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /root/.ivy2/cache
The jars for the packages stored in: /root/.ivy2/jars
org.apache.hadoop#hadoop-aws added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-3ea40dd1-8c9d-4578-b64a-d01e590d83c1;1.0
	confs: [default]
	found org.apache.hadoop#hadoop-aws;3.2.2 in central
	found com.amazonaws#aws-java-sdk-bundle;1.11.563 in central
:: resolution report :: resolve 366ms :: artifacts dl 28ms
	:: modules in use:
	com.amazonaws#aws-java-sdk-bundle;1.11.563 from central in [default]
	org.apache.hadoop#hadoop-aws;3.2.2 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlded|evicted|| number|dwnlded|
	---------------------------------------------------------------------
	|      default     |   2   |   0   |   0   |   0   ||   2   |   0   |
	---------------------------------------------

23/11/13 19:00:12 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


3.3.0


In [4]:
import sagemaker

session = sagemaker.Session()
bucket = session.default_bucket()
print(bucket)

# Create or retrieve a Spark session
spark = SparkSession.builder.appName("ReadS3Parquet").getOrCreate()


sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /root/.config/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /root/.config/sagemaker/config.yaml
sagemaker-us-east-1-711387073580
23/11/13 19:00:21 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


In [5]:
from pyspark.sql.functions import date_format
from pyspark.sql.functions import avg
from pyspark.sql.functions import to_date
from pyspark.sql.functions import date_format
from pyspark.sql.functions import count

# Data

In [50]:
# S3 directory path
s3_directory = f"s3a://{bucket}/project/cleaned/comments/"

# Read all the Parquet files in the directory into a DataFrame
df_comments = spark.read.parquet(s3_directory)

In [8]:
# S3 directory path
s3_directory = f"s3a://{bucket}/project/cleaned/submissions/"

# Read all the Parquet files in the directory into a DataFrame
df_submissions = spark.read.parquet(s3_directory)

23/11/13 16:17:39 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


# Importing Data for other Subreddits 

In [9]:
# Tegveer subreddits
tegveer_bucket = 'sagemaker-us-east-1-433974840707'
s3_directory_tegveer = f"s3a://{tegveer_bucket}/project/cleaned/submissions/"
s3_directory_tegveer_comments = f"s3a://{tegveer_bucket}/project/cleaned/comments/"

# Read all the Parquet files in the directory into a DataFrame
df_submissions_centrist_liberterian = spark.read.parquet(s3_directory_tegveer)
df_comments_centrist_liberterian = spark.read.parquet(s3_directory_tegveer_comments)


In [10]:
# Eric subreddits
eric_bucket = 'sagemaker-us-east-1-395393721134'
s3_directory_eric = f"s3a://{eric_bucket}/project/cleaned/submissions/"
s3_directory_eric_comments = f"s3a://{eric_bucket}/project/cleaned/comments/"

# Read all the Parquet files in the directory into a DataFrame
df_submissions_socialism_economics = spark.read.parquet(s3_directory_eric)
df_submissions_socialism_economics_comments = spark.read.parquet(s3_directory_eric_comments)

In [11]:
# Raunak subreddits

raunak_bucket = 'sagemaker-us-east-1-224518912016'
s3_directory_raunak = f"s3a://{raunak_bucket}/project/cleaned/submissions/"
s3_directory_raunak_comments = f"s3a://{raunak_bucket}/project/cleaned/comments/"

# Read all the Parquet files in the directory into a DataFrame
df_submissions_conservative_finance = spark.read.parquet(s3_directory_raunak)
df_submissions_conservative_finance_comments = spark.read.parquet(s3_directory_raunak_comments)

In [12]:
# My subreddits 

df_submissions_askpol_cmv = df_submissions.filter(df_submissions['subreddit'].isin(['changemyview', 'Ask_Politics']))

# Sample Embeddings

In [13]:
df_submissions_conservative_finance.select(['subreddit' , 'selftext']).show()

+------------+---------+
|   subreddit| selftext|
+------------+---------+
|Conservative|         |
|Conservative|         |
|Conservative|         |
|Conservative|         |
|Conservative|         |
|Conservative|[removed]|
|Conservative|         |
|Conservative|         |
|Conservative|         |
|Conservative|         |
|Conservative|[removed]|
|Conservative|         |
|Conservative|         |
|Conservative|         |
|Conservative|[removed]|
|Conservative|         |
|Conservative|         |
|Conservative|[deleted]|
|Conservative|         |
|Conservative|         |
+------------+---------+
only showing top 20 rows



In [14]:
import pandas as pd

# Assuming df_submissions_conservative_finance is your DataFrame
# Filter out rows where 'selftext' is empty, '[deleted]', or '[removed]'
df_filtered = df_submissions_conservative_finance[
    ~df_submissions_conservative_finance['selftext'].isin(['', '[deleted]', '[removed]'])
]




df_filtered = df_filtered.select(['subreddit' , 'selftext'])

In [15]:
# Subset to test code 
df_filtered_subset = df_filtered.limit(10)

In [16]:
df_filtered_subset.show()

+------------+--------------------+
|   subreddit|            selftext|
+------------+--------------------+
|     finance|This is your safe...|
|     finance|This is your safe...|
|Conservative|If you re a resid...|
|Conservative|Biden is expected...|
|Conservative|I have a family m...|
|     finance|This is your safe...|
|Conservative|I m writing an in...|
|Conservative|Inverse  Scientis...|
|     finance|This is your safe...|
|Conservative|Seriously  What t...|
+------------+--------------------+



In [17]:
from pyspark.ml.feature import Tokenizer, Word2Vec
from pyspark.sql.functions import col

# Step 2: Tokenize the Text
tokenizer = Tokenizer(inputCol="selftext", outputCol="words")
df_tokenized = tokenizer.transform(df_filtered_subset)

# Step 3: Create and Train Word2Vec Model
word2Vec = Word2Vec(vectorSize=100, minCount=0, inputCol="words", outputCol="embeddings")
model = word2Vec.fit(df_tokenized)

# Step 4: Transform the Data
df_with_embeddings = model.transform(df_tokenized)

23/11/13 16:17:55 WARN InstanceBuilder$NativeBLAS: Failed to load implementation from:dev.ludovic.netlib.blas.JNIBLAS
23/11/13 16:17:55 WARN InstanceBuilder$NativeBLAS: Failed to load implementation from:dev.ludovic.netlib.blas.ForeignLinkerBLAS


In [18]:
# Now df_with_embeddings will have a new column "embeddings" with Word2Vec embeddings
df_with_embeddings.select("subreddit", "selftext", "embeddings").show()


+------------+--------------------+--------------------+
|   subreddit|            selftext|          embeddings|
+------------+--------------------+--------------------+
|     finance|This is your safe...|[-0.0010451586257...|
|     finance|This is your safe...|[-0.0010451586257...|
|Conservative|If you re a resid...|[-0.0011812509759...|
|Conservative|Biden is expected...|[-0.0018011155347...|
|Conservative|I have a family m...|[-4.5234596554008...|
|     finance|This is your safe...|[-0.0010451586257...|
|Conservative|I m writing an in...|[-0.0010573752661...|
|Conservative|Inverse  Scientis...|[-0.0011679093748...|
|     finance|This is your safe...|[-0.0010451586257...|
|Conservative|Seriously  What t...|[-0.0014898275501...|
+------------+--------------------+--------------------+



In [19]:
pandas_df_with_embeddings = df_with_embeddings.toPandas()

In [35]:
pandas_df_with_embeddings.to_csv("sample_embeddings.csv")

# Embedding all the data

In [37]:
df_filtered.count()

1566

In [20]:
# Joining all the dataframes 

combined_df = (df_submissions_centrist_liberterian
               .union(df_submissions_socialism_economics)
               .union(df_submissions_conservative_finance)
               .union(df_submissions_askpol_cmv))

combined_df.count()

599908

In [21]:
combined_df = combined_df.select(['subreddit','selftext'])


cleaned_combined_df = combined_df[
    ~combined_df['selftext'].isin(['', '[deleted]', '[removed]'])
]


cleaned_combined_df.show(5)

+-----------+--------------------+
|  subreddit|            selftext|
+-----------+--------------------+
|Libertarian| Fookin libertarians|
|Libertarian|I personally beli...|
|Libertarian|[https://www.vox....|
|Libertarian|It is likely that...|
|Libertarian|Libertarian: an a...|
+-----------+--------------------+
only showing top 5 rows



In [22]:
cleaned_combined_df.count()

39767

In [23]:
cleaned_combined_df.show(5)

+-----------+--------------------+
|  subreddit|            selftext|
+-----------+--------------------+
|Libertarian| Fookin libertarians|
|Libertarian|I personally beli...|
|Libertarian|[https://www.vox....|
|Libertarian|It is likely that...|
|Libertarian|Libertarian: an a...|
+-----------+--------------------+
only showing top 5 rows



In [25]:
# S3 directory path
s3_directory = f"s3a://{bucket}/project/cleaned/"

# Write the DataFrame to S3
cleaned_combined_df.write.format("parquet").save(s3_directory + "combined_submissions")


# Read in Combined Data from S3

In [10]:
s3_directory = f"s3a://{bucket}/project/cleaned/"

# Read the DataFrame from S3
cleaned_combined_df = spark.read.format("parquet").load(s3_directory + "combined_submissions")

In [8]:
cleaned_combined_df.count()

39767

In [15]:
cleaned_combined_df.show()

+------------+--------------------+
|   subreddit|            selftext|
+------------+--------------------+
|Ask_Politics|I did a quick sea...|
|Ask_Politics|Hello, I have a q...|
|changemyview|I keep reading an...|
|changemyview|I am a Marxist, a...|
|changemyview|Exceptions of cou...|
|changemyview|Disclaimer: This ...|
|changemyview|\n\nI love histor...|
|changemyview|So, a bit of a ba...|
|changemyview|Putting up securi...|
|changemyview|Video games have ...|
|changemyview|Before I begin, I...|
|changemyview|&gt; A quick pref...|
|changemyview|As the internet h...|
|changemyview|Everybody likes t...|
|Ask_Politics|Say Democrats wer...|
|changemyview|I understand hiki...|
|changemyview|I can't think of ...|
|changemyview|In this scenario ...|
|Ask_Politics|In 2020, it was r...|
|changemyview|I understand that...|
+------------+--------------------+
only showing top 20 rows



In [22]:
x = cleaned_combined_df.limit(5)

In [12]:
from pyspark.sql import SparkSession
from pyspark.ml.feature import Tokenizer, Word2Vec
from pyspark.sql.functions import monotonically_increasing_id
from pyspark.ml.linalg import VectorUDT
from pyspark.sql.types import StructType, StructField

# Initialize Spark session
spark = SparkSession.builder.appName("word2vec_application").getOrCreate()

# Assuming 'cleaned_combined_df' is your existing Spark DataFrame
df_with_id = cleaned_combined_df.withColumn("id", monotonically_increasing_id())

# Determine the batch size
batch_size = 100

# Calculate the number of batches needed
num_rows = df_with_id.count()
num_batches = (num_rows + batch_size - 1) // batch_size

# Tokenize the text in the 'selftext' column
tokenizer = Tokenizer(inputCol="selftext", outputCol="words")
tokenized_df = tokenizer.transform(df_with_id)

# Define the schema for the result DataFrame including the 'features' column
result_schema = StructType(tokenized_df.schema.fields + [StructField("features", VectorUDT(), True)])

# Initialize an empty DataFrame to store results
result_df = spark.createDataFrame([], result_schema)

# Process each batch
for i in range(num_batches):
    # Extract the batch
    start_id = i * batch_size
    end_id = start_id + batch_size
    batch_df = tokenized_df.filter((tokenized_df.id >= start_id) & (tokenized_df.id < end_id))

    # Apply Word2Vec on the batch
    word2Vec = Word2Vec(vectorSize=100, minCount=0, inputCol="words", outputCol="features")
    model = word2Vec.fit(batch_df)
    batch_result = model.transform(batch_df)

    # Append the processed batch to the result DataFrame
    result_df = result_df.union(batch_result)

# Drop the temporary 'id' column
final_result_df = result_df.drop("id")


23/11/13 19:05:19 WARN InstanceBuilder$NativeBLAS: Failed to load implementation from:dev.ludovic.netlib.blas.JNIBLAS
23/11/13 19:05:19 WARN InstanceBuilder$NativeBLAS: Failed to load implementation from:dev.ludovic.netlib.blas.ForeignLinkerBLAS


IllegalArgumentException: requirement failed: The vocabulary size should be > 0. You may need to check the setting of minCount, which could be large enough to remove all your words in sentences.

In [14]:
result_df.count()

23/11/13 19:38:49 WARN DAGScheduler: Broadcasting large task binary with size 1431.7 KiB


31941

In [15]:
from pyspark.sql.utils import IllegalArgumentException

processed_records = 31941
start_batch_num = processed_records // batch_size

for i in range(start_batch_num, num_batches):
    try:
        # Extract the batch
        start_id = i * batch_size
        end_id = start_id + batch_size
        batch_df = tokenized_df.filter((tokenized_df.id >= start_id) & (tokenized_df.id < end_id))

        # Apply Word2Vec on the batch
        word2Vec = Word2Vec(vectorSize=100, minCount=0, inputCol="words", outputCol="features")
        model = word2Vec.fit(batch_df)
        batch_result = model.transform(batch_df)

        # Append the processed batch to the result DataFrame
        result_df = result_df.union(batch_result)
    except IllegalArgumentException as e:
        print(f"Skipping batch {i} due to error: {e}")

# Drop the temporary 'id' column
final_result_df = result_df.drop("id")


Skipping batch 320 due to error: requirement failed: The vocabulary size should be > 0. You may need to check the setting of minCount, which could be large enough to remove all your words in sentences.


Skipping batch 321 due to error: requirement failed: The vocabulary size should be > 0. You may need to check the setting of minCount, which could be large enough to remove all your words in sentences.


Skipping batch 322 due to error: requirement failed: The vocabulary size should be > 0. You may need to check the setting of minCount, which could be large enough to remove all your words in sentences.


Skipping batch 323 due to error: requirement failed: The vocabulary size should be > 0. You may need to check the setting of minCount, which could be large enough to remove all your words in sentences.


Skipping batch 324 due to error: requirement failed: The vocabulary size should be > 0. You may need to check the setting of minCount, which could be large enough to remove all your words in sentences.


...

Skipping batch 393 due to error: requirement failed: The vocabulary size should be > 0. You may need to check the setting of minCount, which could be large enough to remove all your words in sentences.


Skipping batch 394 due to error: requirement failed: The vocabulary size should be > 0. You may need to check the setting of minCount, which could be large enough to remove all your words in sentences.


Skipping batch 395 due to error: requirement failed: The vocabulary size should be > 0. You may need to check the setting of minCount, which could be large enough to remove all your words in sentences.


Skipping batch 396 due to error: requirement failed: The vocabulary size should be > 0. You may need to check the setting of minCount, which could be large enough to remove all your words in sentences.


Skipping batch 397 due to error: requirement failed: The vocabulary size should be > 0. You may need to check the setting of minCount, which could be large enough to remove all your words in sentences.


In [17]:
final_result_df.show()

23/11/13 19:57:07 WARN DAGScheduler: Broadcasting large task binary with size 4.3 MiB


23/11/13 19:57:08 WARN DAGScheduler: Broadcasting large task binary with size 4.3 MiB


+------------+--------------------+--------------------+--------------------+
|   subreddit|            selftext|               words|            features|
+------------+--------------------+--------------------+--------------------+
|Ask_Politics|I did a quick sea...|[i, did, a, quick...|[0.01834424620681...|
|Ask_Politics|Hello, I have a q...|[hello,, i, have,...|[0.01549940554369...|
|changemyview|I keep reading an...|[i, keep, reading...|[0.01631413909206...|
|changemyview|I am a Marxist, a...|[i, am, a, marxis...|[0.01836980200796...|
|changemyview|Exceptions of cou...|[exceptions, of, ...|[0.01382255466482...|
|changemyview|Disclaimer: This ...|[disclaimer:, thi...|[0.01601292483145...|
|changemyview|\n\nI love histor...|[, , i, love, his...|[0.02111907340269...|
|changemyview|So, a bit of a ba...|[so,, a, bit, of,...|[0.01819114568346...|
|changemyview|Putting up securi...|[putting, up, sec...|[0.01465115683626...|
|changemyview|Video games have ...|[video, games, ha...|[0.01793

In [18]:
s3_directory = f"s3a://{bucket}/project/cleaned/"

# Write the DataFrame to S3
result_df.write.format("parquet").save(s3_directory + "embedded_submissions")

23/11/13 20:00:30 WARN DAGScheduler: Broadcasting large task binary with size 5.4 MiB


In [19]:
# Assuming 'result_df' is your Spark DataFrame
pandas_df = result_df.toPandas()


23/11/13 20:21:40 WARN DAGScheduler: Broadcasting large task binary with size 5.2 MiB


## Save Word2Vec model for later use

In [24]:
model_path = "models/my_word2vec_model"
model.save(model_path)